In [122]:
import numpy as np
import random
import pandas as pd
import math
from random import randint

In [123]:
def _readfile(file):

    # ----- read data -----
    df = pd.read_csv(file)
    df = df.fillna(method = 'ffill')

    # ----- Create Features -----
    X = df.drop(['id','class'], axis=1).copy(deep=False)
    X = (X-X.min())/(X.max()-X.min())

    # ----- Create Desired outputs -----
    Y = df[['class']].copy(deep=False)

    Input = X.to_numpy()
    Output = Y.to_numpy()

    return Input,Output

Input,Output = _readfile('data/wdbc.csv')

In [124]:
def cross_validations_split(shape,folds):
    fold_size = int(shape * folds/100)
    k = 0
    index = []
    for i in range(1,folds+1):
        if i < folds:
            index.append([k,i*fold_size])
        else:
            index.append([k,shape])
        k = i*fold_size
    return index

In [125]:
class GA(object):
    def __init__(self, hiddenSize, inputSize, outputSize):

        # initiate layers
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize

        layers = [self.inputSize] + self.hiddenSize + [self.outputSize]

        # initiate genes
        genes = []
        for i in range(len(layers)-1):
            g = np.random.uniform(-2,2,(layers[i], layers[i+1]))
            genes.append(g)
        self.genes = genes
        

    def feedForward(self, X):
        Output_node = X
        for i, g in enumerate(self.genes):
        
            v = np.dot(Output_node, g)
            Output_node = self.sigmoid(v)

        return Output_node

    def sigmoid(self, s, deriv=False):
        if (deriv == True):
            return s * (1-s)
        return 1/(1 + np.exp(-s))

    def object_funct(self, X, Y):

        seed = randint(1, 100*100)

        np.random.seed(seed)
        np.random.shuffle(X)

        np.random.seed(seed)
        np.random.shuffle(Y)

        sum_err = 0
        for j, input in enumerate(X):
            target = Y[j]
            output = self.feedForward(input)

            sum_err += self._mse(target, output)

        self.fx = 1/math.log(1+sum_err/len(X))
        
        return self.fx

    def _mse(self, target, output):
        return np.average(abs(target - output)**2)


In [126]:
class selection(object):
    def __init__(self, chromosomes):

        # initiate fitness
        self.chromosomes = chromosomes
        self.Max = 1.2
        self.Min = 2-self.Max

        self.P = [0]*len(chromosomes)
    
    # initiate probability
    def prob(self):

        P = []
        N = len(chromosomes)
        Max = self.Max
        Min = self.Min

        for r,_ in enumerate(self.chromosomes):
            p = (Min + (Max - Min) * ((r-1)/(N-1)))/N
            P.append(p)
        self.P = P
        return self.P



# Stochastic universal sampling
#def sus(fx):


In [127]:
chromosomes = []
no_generations = 100
no_chromosomes = 20

In [128]:
# initiate chromosomes P(0)
for i in range(no_chromosomes):
    c = GA([20,10],Input.shape[1],Output.shape[1])
    chromosomes.append(c)

# initiate best objective function
best_chromosome = [float('-inf'),chromosomes[0]]

x_train, y_train = Input,Output

In [130]:
for t in range(no_generations):

    print("----------------------------- P[",t,"] -----------------------------")

    # evaluation objective function
    ranks = []
    for c in chromosomes:

        fx = c.object_funct(x_train, y_train)
        ranks.append([fx,c])

        if fx > best_chromosome[0]:
            print("--- update best fitenss ",round(fx,6)," ---")
            best_chromosome[0] = fx
            best_chromosome[1] = c

    ranks = sorted(ranks, key = lambda x: (x[0]))
    print(selection(ranks).prob())
    break


----------------------------- P[ 0 ] -----------------------------
[0.03894736842105263, 0.04, 0.04105263157894737, 0.04210526315789474, 0.0431578947368421, 0.04421052631578947, 0.045263157894736845, 0.04631578947368421, 0.04736842105263158, 0.04842105263157895, 0.049473684210526316, 0.05052631578947368, 0.05157894736842106, 0.05263157894736842, 0.05368421052631579, 0.05473684210526315, 0.05578947368421052, 0.056842105263157895, 0.05789473684210526, 0.05894736842105262]
